# 📦 Instalação de Dependências

Execute esta célula primeiro para instalar todas as bibliotecas necessárias.

In [1]:
# Instalação das dependências necessárias
!pip install pandas numpy requests matplotlib seaborn jupyter ipython
!pip install pvlib-python scipy scikit-learn

# Verificar instalação
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Todas as dependências foram instaladas com sucesso!")
print(f"📊 Pandas versão: {pd.__version__}")
print(f"🔢 NumPy versão: {np.__version__}")
print(f"🌐 Requests disponível")
print(f"📈 Matplotlib disponível")
print(f"🎨 Seaborn disponível")

ERROR: Could not find a version that satisfies the requirement pvlib-python (from versions: none)
ERROR: No matching distribution found for pvlib-python
✅ Todas as dependências foram instaladas com sucesso!
📊 Pandas versão: 2.3.2
🔢 NumPy versão: 2.3.2
🌐 Requests disponível
📈 Matplotlib disponível
🎨 Seaborn disponível


# Otimizador de Módulos Solares por Localização

Este notebook recebe uma coordenada (latitude, longitude) e testa qual modelo de módulo solar oferece o melhor desempenho para essa localização específica.

## Funcionalidades:
- Análise de irradiação solar via PVGIS
- Comparação de desempenho de múltiplos modelos de módulos
- Consideração de fatores climáticos (temperatura, coeficientes)
- Análise de geração anual e eficiência por modelo
- Recomendação do melhor modelo baseado em múltiplos critérios

In [2]:
# Importações necessárias
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

## 1. Base de Dados de Módulos Solares

Definindo os módulos solares disponíveis no sistema BessPro com suas especificações técnicas.

In [3]:
# Base de dados dos módulos solares do sistema
SOLAR_MODULES = {
    'Jinko Solar JKM550M-7RL4-V': {
        'fabricante': 'Jinko Solar',
        'modelo': 'JKM550M-7RL4-V',
        'potencia_nominal': 550,  # Watts
        'largura_mm': 2278,
        'altura_mm': 1134,
        'vmpp': 41.8,  # V
        'impp': 13.16,  # A
        'voc': 49.7,  # V
        'isc': 13.98,  # A
        'eficiencia': 21.2,  # %
        'temp_coef_pmax': -0.40,  # %/°C
        'temp_coef_voc': -0.27,  # %/°C
        'temp_coef_isc': 0.048,  # %/°C
        'tipo_celula': 'Monocristalino PERC',
        'peso_kg': 27.5
    },
    'Canadian Solar CS3W-540MS': {
        'fabricante': 'Canadian Solar',
        'modelo': 'CS3W-540MS',
        'potencia_nominal': 540,
        'largura_mm': 2261,
        'altura_mm': 1134,
        'vmpp': 41.4,
        'impp': 13.04,
        'voc': 49.8,
        'isc': 13.8,
        'eficiencia': 20.9,
        'temp_coef_pmax': -0.37,
        'temp_coef_voc': -0.26,
        'temp_coef_isc': 0.05,
        'tipo_celula': 'Monocristalino PERC',
        'peso_kg': 27.2
    },
    'Trina Solar TSM-545DEG21C.20': {
        'fabricante': 'Trina Solar',
        'modelo': 'TSM-545DEG21C.20',
        'potencia_nominal': 545,
        'largura_mm': 2279,
        'altura_mm': 1134,
        'vmpp': 41.6,
        'impp': 13.10,
        'voc': 49.9,
        'isc': 13.85,
        'eficiencia': 21.0,
        'temp_coef_pmax': -0.35,
        'temp_coef_voc': -0.25,
        'temp_coef_isc': 0.045,
        'tipo_celula': 'Monocristalino PERC',
        'peso_kg': 27.8
    },
    'Risen Energy RSM144-8-550M': {
        'fabricante': 'Risen Energy',
        'modelo': 'RSM144-8-550M',
        'potencia_nominal': 550,
        'largura_mm': 2278,
        'altura_mm': 1134,
        'vmpp': 41.7,
        'impp': 13.19,
        'voc': 49.8,
        'isc': 13.95,
        'eficiencia': 21.3,
        'temp_coef_pmax': -0.36,
        'temp_coef_voc': -0.26,
        'temp_coef_isc': 0.048,
        'tipo_celula': 'Monocristalino PERC',
        'peso_kg': 27.6
    },
    'JA Solar JAM72D30-550/MB': {
        'fabricante': 'JA Solar',
        'modelo': 'JAM72D30-550/MB',
        'potencia_nominal': 550,
        'largura_mm': 2279,
        'altura_mm': 1134,
        'vmpp': 41.9,
        'impp': 13.13,
        'voc': 50.1,
        'isc': 13.88,
        'eficiencia': 21.1,
        'temp_coef_pmax': -0.38,
        'temp_coef_voc': -0.27,
        'temp_coef_isc': 0.047,
        'tipo_celula': 'Monocristalino PERC',
        'peso_kg': 27.9
    },
    'LONGi Solar LR5-72HTH-545M': {
        'fabricante': 'LONGi Solar',
        'modelo': 'LR5-72HTH-545M',
        'potencia_nominal': 545,
        'largura_mm': 2278,
        'altura_mm': 1134,
        'vmpp': 41.5,
        'impp': 13.13,
        'voc': 49.6,
        'isc': 13.92,
        'eficiencia': 21.0,
        'temp_coef_pmax': -0.35,
        'temp_coef_voc': -0.26,
        'temp_coef_isc': 0.048,
        'tipo_celula': 'Monocristalino PERC',
        'peso_kg': 27.4
    }
}

print(f"📊 Módulos carregados: {len(SOLAR_MODULES)} modelos")
print("\n🔌 Modelos disponíveis:")
for i, (key, module) in enumerate(SOLAR_MODULES.items(), 1):
    print(f"{i:2d}. {module['fabricante']} - {module['modelo']} ({module['potencia_nominal']}W)")

📊 Módulos carregados: 6 modelos

🔌 Modelos disponíveis:
 1. Jinko Solar - JKM550M-7RL4-V (550W)
 2. Canadian Solar - CS3W-540MS (540W)
 3. Trina Solar - TSM-545DEG21C.20 (545W)
 4. Risen Energy - RSM144-8-550M (550W)
 5. JA Solar - JAM72D30-550/MB (550W)
 6. LONGi Solar - LR5-72HTH-545M (545W)


## 2. Função de Consulta PVGIS

Obtém dados de irradiação solar e temperatura ambiente para a localização especificada.

In [4]:
def get_pvgis_data(latitude: float, longitude: float, tilt: int = 20) -> Dict:
    """
    Obtém dados de irradiação solar e temperatura do PVGIS para uma localização.
    
    Args:
        latitude: Latitude da localização
        longitude: Longitude da localização
        tilt: Ângulo de inclinação dos módulos (padrão: 20°)
    
    Returns:
        Dict com dados mensais de irradiação e temperatura
    """
    try:
        # URL da API PVGIS
        url = "https://re.jrc.ec.europa.eu/api/v5_2/PVcalc"
        
        params = {
            'lat': latitude,
            'lon': longitude,
            'raddatabase': 'PVGIS-SARAH2',
            'browser': '0',
            'outputformat': 'json',
            'userhorizon': '1',
            'pvtechchoice': 'crystSi',
            'peakpower': '1',  # 1kWp para normalizar
            'loss': '14',  # Perdas do sistema (14%)
            'mountingplace': 'free',
            'angle': tilt,
            'aspect': '180'  # Face sul
        }
        
        print(f"🌍 Consultando PVGIS para coordenadas: {latitude:.4f}, {longitude:.4f}")
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        
        data = response.json()
        
        # Extrair dados mensais
        monthly_data = data['outputs']['monthly']['fixed']
        
        irradiation_monthly = [month['H(i)_d'] for month in monthly_data]
        temperature_monthly = [month['T2m'] for month in monthly_data]
        
        # Dados anuais
        annual_data = data['outputs']['totals']['fixed']
        annual_irradiation = annual_data['H(i)_y']
        annual_generation = annual_data['E_y']
        
        result = {
            'irradiation_monthly': irradiation_monthly,  # kWh/m²/dia
            'temperature_monthly': temperature_monthly,  # °C
            'annual_irradiation': annual_irradiation,    # kWh/m²/ano
            'annual_generation_per_kwp': annual_generation,  # kWh/kWp/ano
            'location': {
                'latitude': latitude,
                'longitude': longitude,
                'elevation': data['inputs']['location']['elevation']
            }
        }
        
        print(f"✅ Dados PVGIS obtidos com sucesso!")
        print(f"   📍 Elevação: {result['location']['elevation']} m")
        print(f"   ☀️ Irradiação anual: {annual_irradiation:.1f} kWh/m²/ano")
        print(f"   ⚡ Geração específica: {annual_generation:.1f} kWh/kWp/ano")
        
        return result
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erro na consulta PVGIS: {e}")
        return None
    except Exception as e:
        print(f"❌ Erro inesperado: {e}")
        return None

## 3. Calculadora de Desempenho de Módulos

Calcula o desempenho esperado de cada módulo considerando as condições climáticas locais.

In [5]:
def calculate_module_performance(module_data: Dict, pvgis_data: Dict) -> Dict:
    """
    Calcula o desempenho de um módulo específico baseado nos dados climáticos locais.
    
    Args:
        module_data: Especificações do módulo solar
        pvgis_data: Dados climáticos da localização
    
    Returns:
        Dict com métricas de desempenho do módulo
    """
    # Área do módulo em m²
    area_m2 = (module_data['largura_mm'] * module_data['altura_mm']) / 1_000_000
    
    # Densidade de potência (W/m²)
    power_density = module_data['potencia_nominal'] / area_m2
    
    # Cálculos mensais
    monthly_results = []
    total_generation = 0
    
    # Temperatura de referência STC (25°C)
    temp_stc = 25.0
    
    for month in range(12):
        irradiation_daily = pvgis_data['irradiation_monthly'][month]  # kWh/m²/dia
        temperature_avg = pvgis_data['temperature_monthly'][month]  # °C
        
        # Temperatura da célula (aproximação: Temp_ambiente + 25°C sob operação)
        cell_temp = temperature_avg + 25
        
        # Correção de temperatura para potência
        temp_correction = 1 + (module_data['temp_coef_pmax'] / 100) * (cell_temp - temp_stc)
        
        # Potência corrigida por temperatura
        power_corrected = module_data['potencia_nominal'] * temp_correction
        
        # Geração mensal (kWh)
        days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31][month]
        monthly_generation = (irradiation_daily * power_corrected / 1000) * days_in_month
        
        monthly_results.append({
            'month': month + 1,
            'irradiation_daily': irradiation_daily,
            'temperature_avg': temperature_avg,
            'cell_temperature': cell_temp,
            'temp_correction': temp_correction,
            'power_corrected': power_corrected,
            'generation_kwh': monthly_generation
        })
        
        total_generation += monthly_generation
    
    # Métricas de desempenho
    specific_yield = total_generation / (module_data['potencia_nominal'] / 1000)  # kWh/kWp/ano
    performance_ratio = specific_yield / pvgis_data['annual_irradiation']  # Adimensional
    capacity_factor = (total_generation / (module_data['potencia_nominal'] / 1000)) / 8760 * 100  # %
    
    # Eficiência real considerando temperatura
    avg_temp_correction = np.mean([m['temp_correction'] for m in monthly_results])
    real_efficiency = module_data['eficiencia'] * avg_temp_correction
    
    return {
        'module_info': {
            'fabricante': module_data['fabricante'],
            'modelo': module_data['modelo'],
            'potencia_nominal': module_data['potencia_nominal'],
            'eficiencia_nominal': module_data['eficiencia'],
            'area_m2': area_m2,
            'power_density': power_density
        },
        'performance': {
            'annual_generation_kwh': total_generation,
            'specific_yield_kwh_kwp': specific_yield,
            'performance_ratio': performance_ratio,
            'capacity_factor_percent': capacity_factor,
            'real_efficiency_percent': real_efficiency,
            'avg_temp_correction': avg_temp_correction
        },
        'monthly_data': monthly_results
    }

## 4. Analisador de Múltiplos Módulos

Compara o desempenho de todos os módulos disponíveis para uma localização.

In [6]:
def analyze_all_modules(latitude: float, longitude: float) -> pd.DataFrame:
    """
    Analisa todos os módulos disponíveis para uma localização específica.
    
    Args:
        latitude: Latitude da localização
        longitude: Longitude da localização
    
    Returns:
        DataFrame com comparação de desempenho de todos os módulos
    """
    print(f"\n🔍 Iniciando análise para coordenadas: {latitude:.4f}, {longitude:.4f}")
    print("="*70)
    
    # Obter dados climáticos
    pvgis_data = get_pvgis_data(latitude, longitude)
    if not pvgis_data:
        print("❌ Falha ao obter dados climáticos. Análise cancelada.")
        return None
    
    print("\n🧮 Calculando desempenho de todos os módulos...")
    
    results = []
    
    for module_name, module_data in SOLAR_MODULES.items():
        print(f"   ⚡ Processando: {module_data['fabricante']} {module_data['modelo']}")
        
        performance = calculate_module_performance(module_data, pvgis_data)
        
        # Compilar resultados
        result_row = {
            'Fabricante': performance['module_info']['fabricante'],
            'Modelo': performance['module_info']['modelo'],
            'Potência_Nominal_W': performance['module_info']['potencia_nominal'],
            'Eficiência_Nominal_%': performance['module_info']['eficiencia_nominal'],
            'Área_m²': round(performance['module_info']['area_m2'], 2),
            'Densidade_Potência_W/m²': round(performance['module_info']['power_density'], 1),
            'Geração_Anual_kWh': round(performance['performance']['annual_generation_kwh'], 1),
            'Rendimento_Específico_kWh/kWp': round(performance['performance']['specific_yield_kwh_kwp'], 1),
            'Taxa_Performance': round(performance['performance']['performance_ratio'], 3),
            'Fator_Capacidade_%': round(performance['performance']['capacity_factor_percent'], 2),
            'Eficiência_Real_%': round(performance['performance']['real_efficiency_percent'], 2),
            'Correção_Temperatura': round(performance['performance']['avg_temp_correction'], 4),
            'Coef_Temp_Pmax_%/°C': module_data['temp_coef_pmax']
        }
        
        results.append(result_row)
    
    # Criar DataFrame e ordenar por geração anual
    df = pd.DataFrame(results)
    df = df.sort_values('Geração_Anual_kWh', ascending=False).reset_index(drop=True)
    
    print(f"\n✅ Análise concluída para {len(results)} módulos!")
    print(f"📍 Localização: {latitude:.4f}, {longitude:.4f}")
    print(f"🌡️ Temperatura média anual: {np.mean(pvgis_data['temperature_monthly']):.1f}°C")
    print(f"☀️ Irradiação anual: {pvgis_data['annual_irradiation']:.1f} kWh/m²/ano")
    
    return df, pvgis_data

## 5. Sistema de Recomendação

Analisa múltiplos critérios para recomendar o melhor módulo.

In [7]:
def recommend_best_module(df: pd.DataFrame, criteria_weights: Dict = None) -> Dict:
    """
    Recomenda o melhor módulo baseado em múltiplos critérios ponderados.
    
    Args:
        df: DataFrame com resultados de performance
        criteria_weights: Pesos para cada critério (opcional)
    
    Returns:
        Dict com recomendação e pontuação detalhada
    """
    # Pesos padrão para os critérios (somam 1.0)
    if criteria_weights is None:
        criteria_weights = {
            'geracao_anual': 0.35,      # 35% - Geração absoluta
            'eficiencia_real': 0.25,    # 25% - Eficiência real
            'rendimento_especifico': 0.20,  # 20% - kWh/kWp
            'densidade_potencia': 0.10, # 10% - Aproveitamento de espaço
            'taxa_performance': 0.10    # 10% - Performance ratio
        }
    
    print("\n🎯 Calculando pontuação multi-critério...")
    print("\n📊 Pesos dos critérios:")
    for criterio, peso in criteria_weights.items():
        print(f"   • {criterio.replace('_', ' ').title()}: {peso*100:.0f}%")
    
    # Normalizar critérios (0-100 pontos cada)
    df_scored = df.copy()
    
    # Critérios onde maior é melhor
    for col, weight in [
        ('Geração_Anual_kWh', criteria_weights['geracao_anual']),
        ('Eficiência_Real_%', criteria_weights['eficiencia_real']),
        ('Rendimento_Específico_kWh/kWp', criteria_weights['rendimento_especifico']),
        ('Densidade_Potência_W/m²', criteria_weights['densidade_potencia']),
        ('Taxa_Performance', criteria_weights['taxa_performance'])
    ]:
        # Normalizar para 0-100
        min_val = df[col].min()
        max_val = df[col].max()
        normalized = (df[col] - min_val) / (max_val - min_val) * 100
        
        # Aplicar peso
        score_col = f"Score_{col.split('_')[0]}"
        df_scored[score_col] = normalized * weight * 100  # *100 para ficar 0-100 final
    
    # Pontuação total
    score_columns = [col for col in df_scored.columns if col.startswith('Score_')]
    df_scored['Pontuação_Total'] = df_scored[score_columns].sum(axis=1)
    
    # Ordenar por pontuação
    df_scored = df_scored.sort_values('Pontuação_Total', ascending=False).reset_index(drop=True)
    
    # Melhor módulo
    best_module = df_scored.iloc[0]
    
    # Análise detalhada do melhor
    recommendation = {
        'melhor_modulo': {
            'fabricante': best_module['Fabricante'],
            'modelo': best_module['Modelo'],
            'pontuacao_total': round(best_module['Pontuação_Total'], 1)
        },
        'performance': {
            'geracao_anual_kwh': best_module['Geração_Anual_kWh'],
            'rendimento_especifico': best_module['Rendimento_Específico_kWh/kWp'],
            'eficiencia_real': best_module['Eficiência_Real_%'],
            'densidade_potencia': best_module['Densidade_Potência_W/m²'],
            'taxa_performance': best_module['Taxa_Performance']
        },
        'ranking_completo': df_scored[['Fabricante', 'Modelo', 'Pontuação_Total', 
                                       'Geração_Anual_kWh', 'Eficiência_Real_%']].head()
    }
    
    print(f"\n🏆 MELHOR MÓDULO RECOMENDADO:")
    print(f"   📋 {best_module['Fabricante']} - {best_module['Modelo']}")
    print(f"   🎯 Pontuação: {best_module['Pontuação_Total']:.1f}/100")
    print(f"   ⚡ Geração anual: {best_module['Geração_Anual_kWh']:.1f} kWh")
    print(f"   📊 Eficiência real: {best_module['Eficiência_Real_%']:.2f}%")
    print(f"   📈 Rendimento específico: {best_module['Rendimento_Específico_kWh/kWp']:.1f} kWh/kWp")
    
    return recommendation, df_scored

## 6. Visualizações

Funções para criar gráficos comparativos dos módulos.

In [8]:
def create_performance_charts(df: pd.DataFrame, location_info: str):
    """
    Cria gráficos comparativos de desempenho dos módulos.
    """
    # Configurar subplots
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'Análise Comparativa de Módulos Solares - {location_info}', 
                fontsize=16, fontweight='bold')
    
    # 1. Geração Anual
    ax1 = axes[0, 0]
    bars1 = ax1.barh(df['Modelo'], df['Geração_Anual_kWh'], 
                     color=sns.color_palette("viridis", len(df)))
    ax1.set_title('Geração Anual (kWh)', fontweight='bold')
    ax1.set_xlabel('kWh/ano')
    
    # Adicionar valores nas barras
    for i, (bar, value) in enumerate(zip(bars1, df['Geração_Anual_kWh'])):
        ax1.text(value + 1, i, f'{value:.0f}', 
                ha='left', va='center', fontsize=9)
    
    # 2. Eficiência Real
    ax2 = axes[0, 1]
    bars2 = ax2.barh(df['Modelo'], df['Eficiência_Real_%'], 
                     color=sns.color_palette("plasma", len(df)))
    ax2.set_title('Eficiência Real (%)', fontweight='bold')
    ax2.set_xlabel('Eficiência (%)')
    
    for i, (bar, value) in enumerate(zip(bars2, df['Eficiência_Real_%'])):
        ax2.text(value + 0.05, i, f'{value:.2f}%', 
                ha='left', va='center', fontsize=9)
    
    # 3. Rendimento Específico
    ax3 = axes[0, 2]
    bars3 = ax3.barh(df['Modelo'], df['Rendimento_Específico_kWh/kWp'], 
                     color=sns.color_palette("cividis", len(df)))
    ax3.set_title('Rendimento Específico (kWh/kWp)', fontweight='bold')
    ax3.set_xlabel('kWh/kWp/ano')
    
    for i, (bar, value) in enumerate(zip(bars3, df['Rendimento_Específico_kWh/kWp'])):
        ax3.text(value + 5, i, f'{value:.0f}', 
                ha='left', va='center', fontsize=9)
    
    # 4. Densidade de Potência
    ax4 = axes[1, 0]
    bars4 = ax4.barh(df['Modelo'], df['Densidade_Potência_W/m²'], 
                     color=sns.color_palette("rocket", len(df)))
    ax4.set_title('Densidade de Potência (W/m²)', fontweight='bold')
    ax4.set_xlabel('W/m²')
    
    for i, (bar, value) in enumerate(zip(bars4, df['Densidade_Potência_W/m²'])):
        ax4.text(value + 2, i, f'{value:.0f}', 
                ha='left', va='center', fontsize=9)
    
    # 5. Fator de Capacidade
    ax5 = axes[1, 1]
    bars5 = ax5.barh(df['Modelo'], df['Fator_Capacidade_%'], 
                     color=sns.color_palette("mako", len(df)))
    ax5.set_title('Fator de Capacidade (%)', fontweight='bold')
    ax5.set_xlabel('Fator de Capacidade (%)')
    
    for i, (bar, value) in enumerate(zip(bars5, df['Fator_Capacidade_%'])):
        ax5.text(value + 0.05, i, f'{value:.2f}%', 
                ha='left', va='center', fontsize=9)
    
    # 6. Taxa de Performance
    ax6 = axes[1, 2]
    bars6 = ax6.barh(df['Modelo'], df['Taxa_Performance'], 
                     color=sns.color_palette("flare", len(df)))
    ax6.set_title('Taxa de Performance', fontweight='bold')
    ax6.set_xlabel('Performance Ratio')
    
    for i, (bar, value) in enumerate(zip(bars6, df['Taxa_Performance'])):
        ax6.text(value + 0.002, i, f'{value:.3f}', 
                ha='left', va='center', fontsize=9)
    
    # Ajustar layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    plt.show()

def create_ranking_chart(df_scored: pd.DataFrame):
    """
    Cria gráfico de ranking com pontuação total.
    """
    plt.figure(figsize=(12, 8))
    
    # Criar gráfico de barras horizontal
    colors = sns.color_palette("RdYlGn", len(df_scored))
    bars = plt.barh(range(len(df_scored)), df_scored['Pontuação_Total'], color=colors)
    
    # Configurar eixos
    plt.yticks(range(len(df_scored)), 
              [f"{row['Fabricante']}\n{row['Modelo']}" for _, row in df_scored.iterrows()])
    plt.xlabel('Pontuação Total', fontsize=12, fontweight='bold')
    plt.title('Ranking de Módulos Solares - Análise Multi-Critério', 
             fontsize=14, fontweight='bold')
    
    # Adicionar valores nas barras
    for i, (bar, value) in enumerate(zip(bars, df_scored['Pontuação_Total'])):
        plt.text(value + 0.5, i, f'{value:.1f}', 
                ha='left', va='center', fontweight='bold')
    
    # Destacar o melhor
    bars[0].set_color('gold')
    bars[0].set_edgecolor('darkgoldenrod')
    bars[0].set_linewidth(2)
    
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 7. Interface Principal

Execute esta célula inserindo as coordenadas desejadas para análise.

In [ ]:
# =============================================================================
# 🌍 CONFIGURE SUAS COORDENADAS AQUI:
# =============================================================================

# Exemplos de coordenadas (descomente uma linha ou insira suas próprias):

# São Paulo - SP
latitude, longitude = -23.5505, -46.6333

# Rio de Janeiro - RJ
# latitude, longitude = -22.9068, -43.1729

# Brasília - DF
# latitude, longitude = -15.7975, -47.8919

# Belo Horizonte - MG
# latitude, longitude = -19.9191, -43.9378

# Fortaleza - CE
# latitude, longitude = -3.7319, -38.5267

# Porto Alegre - RS
# latitude, longitude = -30.0346, -51.2177

# Suas coordenadas personalizadas:
# latitude, longitude = SEU_LAT, SEU_LON

print(f"🎯 ANÁLISE DE OTIMIZAÇÃO DE MÓDULOS SOLARES")
print(f"="*50)
print(f"📍 Coordenadas selecionadas: {latitude:.4f}, {longitude:.4f}")

## 8. Executar Análise Completa

Executa toda a análise e gera os resultados.

In [ ]:
# Executar análise completa
try:
    # Analisar todos os módulos
    results_df, pvgis_data = analyze_all_modules(latitude, longitude)
    
    if results_df is not None:
        # Exibir resultados básicos
        print("\n📊 RESULTADOS ORDENADOS POR GERAÇÃO ANUAL:")
        print("="*70)
        display(results_df[['Fabricante', 'Modelo', 'Potência_Nominal_W', 
                           'Geração_Anual_kWh', 'Eficiência_Real_%', 
                           'Rendimento_Específico_kWh/kWp']].round(2))
        
        # Sistema de recomendação
        recommendation, scored_df = recommend_best_module(results_df)
        
        # Mostrar ranking final
        print("\n🏆 RANKING FINAL - ANÁLISE MULTI-CRITÉRIO:")
        print("="*70)
        display(scored_df[['Fabricante', 'Modelo', 'Pontuação_Total', 
                          'Geração_Anual_kWh', 'Eficiência_Real_%']].round(2))
        
        # Salvar resultados detalhados
        location_name = f"lat_{latitude:.3f}_lon_{longitude:.3f}"
        results_df.to_csv(f'module_analysis_{location_name}.csv', index=False)
        print(f"\n💾 Resultados salvos em: module_analysis_{location_name}.csv")
        
    else:
        print("❌ Falha na análise. Verifique as coordenadas e conexão com internet.")
        
except Exception as e:
    print(f"❌ Erro durante a análise: {e}")

## 9. Visualizações Gráficas

Gera gráficos comparativos dos resultados.

In [ ]:
# Criar visualizações se a análise foi bem-sucedida
if 'results_df' in locals() and results_df is not None:
    
    location_info = f"Lat: {latitude:.3f}, Lon: {longitude:.3f}"
    
    # Gráficos de performance
    print("📈 Gerando gráficos de performance...")
    create_performance_charts(results_df, location_info)
    
    # Gráfico de ranking
    print("🏆 Gerando gráfico de ranking...")
    create_ranking_chart(scored_df)
    
    # Estatísticas adicionais
    print("\n📊 ESTATÍSTICAS RESUMO:")
    print(f"   • Melhor geração anual: {results_df['Geração_Anual_kWh'].max():.1f} kWh")
    print(f"   • Menor geração anual: {results_df['Geração_Anual_kWh'].min():.1f} kWh")
    print(f"   • Diferença: {results_df['Geração_Anual_kWh'].max() - results_df['Geração_Anual_kWh'].min():.1f} kWh ({((results_df['Geração_Anual_kWh'].max() / results_df['Geração_Anual_kWh'].min() - 1) * 100):.1f}%)")
    
    print(f"   • Melhor eficiência real: {results_df['Eficiência_Real_%'].max():.2f}%")
    print(f"   • Menor eficiência real: {results_df['Eficiência_Real_%'].min():.2f}%")
    
    print(f"   • Temperatura média anual: {np.mean(pvgis_data['temperature_monthly']):.1f}°C")
    print(f"   • Correção média de temperatura: {results_df['Correção_Temperatura'].mean():.3f}")
    
else:
    print("⚠️ Nenhum dado disponível para gerar gráficos.")

## 10. Análise de Sensibilidade (Opcional)

Permite testar diferentes pesos nos critérios de avaliação.

In [ ]:
# Análise de sensibilidade com diferentes pesos
if 'results_df' in locals() and results_df is not None:
    
    print("\n🧪 ANÁLISE DE SENSIBILIDADE - DIFERENTES CENÁRIOS:")
    print("="*70)
    
    scenarios = {
        'Foco em Geração': {
            'geracao_anual': 0.60, 'eficiencia_real': 0.20, 
            'rendimento_especifico': 0.10, 'densidade_potencia': 0.05, 
            'taxa_performance': 0.05
        },
        'Foco em Eficiência': {
            'geracao_anual': 0.20, 'eficiencia_real': 0.50, 
            'rendimento_especifico': 0.15, 'densidade_potencia': 0.10, 
            'taxa_performance': 0.05
        },
        'Foco em Espaço': {
            'geracao_anual': 0.25, 'eficiencia_real': 0.20, 
            'rendimento_especifico': 0.15, 'densidade_potencia': 0.35, 
            'taxa_performance': 0.05
        },
        'Balanceado': {
            'geracao_anual': 0.35, 'eficiencia_real': 0.25, 
            'rendimento_especifico': 0.20, 'densidade_potencia': 0.10, 
            'taxa_performance': 0.10
        }
    }
    
    sensitivity_results = {}
    
    for scenario_name, weights in scenarios.items():
        print(f"\n🎭 Cenário: {scenario_name}")
        rec, df_temp = recommend_best_module(results_df, weights)
        
        winner = rec['melhor_modulo']
        sensitivity_results[scenario_name] = {
            'fabricante': winner['fabricante'],
            'modelo': winner['modelo'],
            'pontuacao': winner['pontuacao_total']
        }
        
        print(f"   🏆 Vencedor: {winner['fabricante']} {winner['modelo']} (Score: {winner['pontuacao_total']:.1f})")
    
    # Resumo da análise de sensibilidade
    print("\n📋 RESUMO DA ANÁLISE DE SENSIBILIDADE:")
    sensitivity_df = pd.DataFrame(sensitivity_results).T
    display(sensitivity_df)
    
    # Contar vitórias por módulo
    winner_counts = {}
    for scenario, result in sensitivity_results.items():
        full_name = f"{result['fabricante']} {result['modelo']}"
        winner_counts[full_name] = winner_counts.get(full_name, 0) + 1
    
    print("\n🏅 MÓDULOS MAIS VERSÁTEIS (vitórias em diferentes cenários):")
    for module, wins in sorted(winner_counts.items(), key=lambda x: x[1], reverse=True):
        print(f"   • {module}: {wins} vitória(s)")

else:
    print("⚠️ Execute primeiro a análise principal para usar a análise de sensibilidade.")

## 📝 Conclusões e Recomendações

### Como Usar Este Notebook:

1. **Configure as coordenadas** na Seção 7
2. **Execute todas as células** sequencialmente
3. **Analise os resultados** nas tabelas e gráficos
4. **Considere diferentes cenários** na análise de sensibilidade

### Critérios de Avaliação:

- **Geração Anual**: Quantidade total de energia produzida
- **Eficiência Real**: Eficiência considerando temperatura local
- **Rendimento Específico**: kWh produzidos por kWp instalado
- **Densidade de Potência**: Aproveitamento de espaço (W/m²)
- **Taxa de Performance**: Qualidade do módulo em condições reais

### Fatores Considerados:

- ✅ Irradiação solar local (PVGIS)
- ✅ Temperatura ambiente média
- ✅ Coeficientes de temperatura dos módulos
- ✅ Especificações técnicas reais
- ✅ Análise multi-critério ponderada

### Limitações:

- Análise baseada apenas em dados técnicos
- Não considera aspectos econômicos (preço, disponibilidade)
- Temperatura da célula é estimada (Tamb + 25°C)
- Não considera sombreamento ou orientação específica

---

**🎯 Este notebook fornece uma base sólida para escolha técnica de módulos solares baseada na localização geográfica e condições climáticas locais.**